# HuggingFace run_clm.py

In [4]:
! pip install kaggle
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download -d paultimothymooney/poetry

  0% 0.00/2.00M [00:00<?, ?B/s]
100% 2.00M/2.00M [00:00<00:00, 46.1MB/s]


In [6]:
! unzip poetry.zip

Archive:  poetry.zip
  inflating: Kanye_West.txt          
  inflating: Lil_Wayne.txt           
  inflating: adele.txt               
  inflating: al-green.txt            
  inflating: alicia-keys.txt         
  inflating: amy-winehouse.txt       
  inflating: beatles.txt             
  inflating: bieber.txt              
  inflating: bjork.txt               
  inflating: blink-182.txt           
  inflating: bob-dylan.txt           
  inflating: bob-marley.txt          
  inflating: britney-spears.txt      
  inflating: bruce-springsteen.txt   
  inflating: bruno-mars.txt          
  inflating: cake.txt                
  inflating: dickinson.txt           
  inflating: disney.txt              
  inflating: dj-khaled.txt           
  inflating: dolly-parton.txt        
  inflating: dr-seuss.txt            
  inflating: drake.txt               
  inflating: eminem.txt              
  inflating: janisjoplin.txt         
  inflating: jimi-hendrix.txt        
  inflating: johnny-cash.txt 

In [18]:
!pip install transformers
!pip install git+https://github.com/huggingface/transformers
!pip install tensorboard
!pip install datasets

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-q6q1r5hb
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-q6q1r5hb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 325 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 38.9 MB/s 
     |████████████████████████████████| 212 kB 54.1 MB/s 
     |████████████████████████████████| 136 kB 42.1 MB/s 
     |████████████████████████████████| 127 kB 40.5 MB/s 
     |████████████████████████████████| 144 kB 39.3 MB/s 
     |████████████████████████████████| 94 kB 2.3 MB/s 
     |████████████████████████████████| 271 kB 46.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into acco

In [21]:
import os, sys, re, time, datetime
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

import pandas as pd
from sklearn.model_selection import train_test_split

import datasets

In [10]:
data = []
for filename in os.listdir():
  if '.txt' in filename:
    with open(filename, 'r') as f:
      data += f.readlines()

In [11]:
train_test_ratio = 0.9
train_valid_ratio = 0.9
train_full, test = train_test_split(data, train_size = train_test_ratio, random_state = 1)
train, val = train_test_split(train_full, train_size = train_valid_ratio, random_state = 1)

In [12]:
len(train), len(val), len(test)

(151541, 16838, 18709)

In [13]:
def build_dataset(lyrics, dest_path):
  f = open(dest_path, 'w')
  data = ''
  phrase = ''
  for lyric in lyrics:
    if lyric == '\n':
      if phrase != '':
        data += '<BOS> ' + phrase + ' <EOS>\n'
        phrase = ''
      else: continue
    else:
      lyric_concat = re.sub(r"\n", ". ", lyric)
      lyric_cleaned = re.sub(r"(\t)+", " ", lyric_concat)
      phrase += lyric_cleaned
        
  f.write(data)
  f.close()
  return data

In [14]:
train = build_dataset(train, "train.txt")
val = build_dataset(val, "val.txt")
test = build_dataset(test, "test.txt")

In [15]:
! mkdir -p scripts
! wget "https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py"
! wget "https://raw.githubusercontent.com/itsuncheng/fine-tuning-GPT2/master/run_generation.py"
! scp "run_clm.py" "scripts/run_clm.py"
! scp "run_generation.py" "scripts/run_generation.py"

--2022-04-28 19:01:27--  https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24515 (24K) [text/plain]
Saving to: ‘run_clm.py’

run_clm.py          100%[===================>]  23.94K  --.-KB/s    in 0.001s  

2022-04-28 19:01:27 (18.6 MB/s) - ‘run_clm.py’ saved [24515/24515]

--2022-04-28 19:01:27--  https://raw.githubusercontent.com/itsuncheng/fine-tuning-GPT2/master/run_generation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Le

In [79]:
! rm -fr models
! mkdir models

In [24]:
!python3 run_clm.py \
    --model_name_or_path gpt2 \
    --train_file 'train.txt' \
    --validation_file 'val.txt' \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --do_train \
    --do_eval \
    --output_dir models \
    --overwrite_output_dir

04/28/2022 19:19:08 - WARNING - __main__ - Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
04/28/2022 19:19:08 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>

# Hugging Face Artists

In [ ]:
! pip install transformer

In [25]:
from transformers import pipeline
generator = pipeline('text-generation',
                     model='huggingartists/muse')
generator("I am feeling ", num_return_sequences=5)

Downloading:   0%|          | 0.00/935 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/236 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I am feeling \nIll never let you down \nAnd I am feeling \nReckless, yeah\nYou are so beautiful \nYoure so beautiful to me\nI just cant let you go\nAnd I am feeling \nIm feeling \nMy mind blowing\nI just cant let you go\nAnd I am feeling \nIm feeling \nIm feeling \nI just cant let you go\nAnd I am feeling \nIm feeling \nI just cant let you go\nI cant get you out of my mind\nI cant get you out of my mind\nIll make you feel \nI want to please you, but I cant\nAnd I am feeling \nIm feeling \nIm feeling Youre such a mess and I cant\nWish I could, but I cant\nLet your inhibitions get the better of me\nLet your inhibitions get the better of me\nIll make you feel \nI want to please you,'},
 {'generated_text': 'I am feeling \nYou make me aroused\nYour scent is on me\nI want you to love me\nMake me obey\nMake me obey\nYou make me want to obey\nDont you know that I adore you?\nYou make me aroused\nYour scent is on me\nI want you to love me\nMake me obey\nMake me obey\nDont y

In [26]:
generator("everything is going to be okay", num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'everything is going to be okay\nI can feel it\nI cant get it right\nYou can see it\nI can feel it\nI can feel it\nI can feel it\nI can feel itDont you just forget me?\nI never had the chance to feel it\nAnd you never could, you could never dream of it\nAnd you could never love me\nAnd you could never love me\nYou could never love me\nDont you just forget me?\nI never had the time to feel it\nAnd you never could, you could never dream of it\nAnd you could never love me\nAnd you could never love meDont you just forget me?\nDont you just forget me?\nI never had the time to feel it\nAnd you never could, you could never dream of it\nAnd you could never love meDont you just forget me?\nI never had the time to feel it\nAnd you never could, you could'},
 {'generated_text': 'everything is going to be okay\nNo one can see or hear us anymore\nThey pretend to be strangers\nBut all the love they bring\nAnd the scars on our souls\nAnd our minds forever fracturedI want to run\nI 

In [27]:
generator = pipeline('text-generation',
                     model='huggingartists/bob-dylan')
generator("I am feeling good", num_return_sequences=5)

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I am feeling good, I am a state trooper\nIm going back to New Orleans, baby, I can see my baby tonight\nI will be back on this coast by the next ship\nI will always be the same, Im sticking with you\nStay up-doin the things you said and you can always do\nYou wont have to feel bad to be good, dont you have to feel bad to be true\nIll be back\nAll right, all right\nLoving you is one thing\nThat every man must keep\nAll right, all right\nHe shouldnt be ashamed of all that you see, all that he takes\nHe shouldnt be proud of you that much\nFor your talent is not to be honed in vain\nIt be glorified in the end\nAll right, all right\nTake off your mask, give it off\nYou wont be a hit\nAll right, all right\nLoving you is one thing\nThat every man must keep'},
 {'generated_text': 'I am feeling good\nHeading out for the east coast\nLord knows Ive paid some dues getting through\nLiving down in New Orleans\nWith my woman of the lowlands\nI met some pretty boys\nAnd they jumpe

In [83]:
eminem = pipeline('text-generation',
                     model='huggingartists/eminem')
eminem("I woke up too late today", num_return_sequences=5)

Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/253 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I woke up too late today\nShe said, “I dont know where I’m goin”\nI said, “here I’m goin”\nShe said, “I dont know where I’m goin”\nI said, “here I’m goin”\nThirty degrees, way too cold, so hold up, dear\nThis is your song, youll never hear it again\nBut youll sing it on your next song, whenever you have the mic\nBounce now, hands up, you know how we do\nWe make the club jump everywhere we go\nIts no secret, everybody know\nWhen Dres involved, there was plenty money involved\nAnd plenty honeys involved, the sunny Sundays and palm trees\nCali, everyday its just another party from the valleys\nAll the way to them 8 Mile alleys, lets rally\nHands up for the grand finale\nNow'},
 {'generated_text': 'I woke up too late today, shes acting weird\nI should call her et cetera, shes goin in your den\nIts hot, Ho think its hot? Ho, chill, act cool\nHad to look out for your next excuse to be at an all-time low\nI smoke a fat pound of grass, fall on my head\nWith a seedling in m